<a href="https://colab.research.google.com/github/haaraamk/DLforAI_class01/blob/main/%EA%B8%B0%EB%A7%90%20%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 파일 직접 로컬에서 불러오기
from google.colab import files
uploaded = files.upload()

#불러오기
file_name = list(uploaded.keys())[0]
test_features = pd.read_csv(file_name)

Saving test_features.csv to test_features (2).csv
Saving train_features.csv to train_features (1).csv
Saving train_labels.csv to train_labels (1).csv


In [5]:
from google.colab import drive
drive.mount('/content/drive')

dpath = "/content/drive/MyDrive/CAU_BURGER/data/"

import pandas as pd

train_features  = pd.read_csv(dpath + "train_features.csv")
test_features  = pd.read_csv(dpath + "test_features.csv")
train_labels = pd.read_csv(dpath + "train_labels.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
train_features.head()
# 이미지 ID, 파일 경로, site ID

,id,filepath,site
0,ZJ000000,train_features/ZJ000000.jpg,S0120
1,ZJ000001,train_features/ZJ000001.jpg,S0069
2,ZJ000002,train_features/ZJ000002.jpg,S0009
3,ZJ000003,train_features/ZJ000003.jpg,S0008
4,ZJ000004,train_features/ZJ000004.jpg,S0036


In [7]:
test_features.head()

,id,filepath,site
0,ZJ016488,test_features/ZJ016488.jpg,S0082
1,ZJ016489,test_features/ZJ016489.jpg,S0040
2,ZJ016490,test_features/ZJ016490.jpg,S0040
3,ZJ016491,test_features/ZJ016491.jpg,S0041
4,ZJ016492,test_features/ZJ016492.jpg,S0040


In [8]:
train_labels.head()

,id,antelope_duiker,bird,blank,civet_genet,hog,leopard,monkey_prosimian,rodent
0,ZJ000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ZJ000001,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,ZJ000002,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ZJ000003,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,ZJ000004,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [32]:
species_labels = sorted(train_labels.columns.unique())
species_labels

['antelope_duiker',
 'bird',
 'blank',
 'civet_genet',
 'hog',
 'id',
 'leopard',
 'monkey_prosimian',
 'rodent']

In [36]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

In [20]:
import pandas as pd
import numpy as np

# sum()과 sort_values() 메서드를 사용하여 필요한 계산을 수행합니다.
result = numeric_labels.sum().sort_values(ascending=False)
result
#종의 분

monkey_prosimian    2492.0
antelope_duiker     2474.0
civet_genet         2423.0
leopard             2254.0
blank               2213.0
rodent              2013.0
bird                1641.0
hog                  978.0
dtype: float64

In [24]:
from sklearn.model_selection import train_test_split

frac = 0.5

y = train_labels.sample(frac=frac, random_state=1)
x = train_features.loc[y.index].filepath.to_frame()

# note that we are casting the species labels to an indicator/dummy matrix
x_train, x_eval, y_train, y_eval = train_test_split(x, y, test_size=0.25, random_state=1)

In [27]:
x_train.head()
#왜 ID가 안들어갔는가 질문

,filepath
3708,train_features/ZJ003708.jpg
13679,train_features/ZJ013679.jpg
15920,train_features/ZJ015920.jpg
3788,train_features/ZJ003788.jpg
3694,train_features/ZJ003694.jpg


In [26]:
y_train.head()

,id,antelope_duiker,bird,blank,civet_genet,hog,leopard,monkey_prosimian,rodent
3708,ZJ003708,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
13679,ZJ013679,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
15920,ZJ015920,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3788,ZJ003788,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3694,ZJ003694,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
x_train.shape, y_train.shape, x_eval.shape, y_eval.shape
#((6183, 1), (6183, 8), (2061, 1), (2061, 8)) -밴치마크의 결과

((6183, 1), (6183, 9), (2061, 1), (2061, 9))

In [29]:
#train 및 eval 세트에서 종의 상대적 분포가 대략 유사한지 확인
split_pcts = pd.DataFrame(
    {
        "train": y_train.idxmax(axis=1).value_counts(normalize=True),
        "eval": y_eval.idxmax(axis=1).value_counts(normalize=True),
    }
)
print("Species percentages by split")
(split_pcts.fillna(0) * 100).astype(int)

TypeError: ignored

In [37]:
#모델 구축
import torch
from torch.utils.data import Dataset
from torchvision import transforms


class ImagesDataset(Dataset):
    """Reads in an image, transforms pixel values, and serves
    a dictionary containing the image id, image tensors, and label.
    """

    def __init__(self, x_df, y_df=None):
        self.data = x_df
        self.label = y_df
        self.transform = transforms.Compose(
            [
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)
                ),
            ]
        )

    def __getitem__(self, index):
        image = Image.open(self.data.iloc[index]["filepath"]).convert("RGB")
        image = self.transform(image)
        image_id = self.data.index[index]
        # if we don't have labels (e.g. for test set) just return the image and image id
        if self.label is None:
            sample = {"image_id": image_id, "image": image}
        else:
            label = torch.tensor(self.label.iloc[index].values, 
                                 dtype=torch.float)
            sample = {"image_id": image_id, "image": image, "label": label}
        return sample

    def __len__(self):
        return len(self.data)

In [38]:
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.imagenet_utils import preprocess_input

class ImagesDataset(Sequence):
    def __init__(self, x_df, y_df=None, batch_size=32):
        self.x_df = x_df
        self.y_df = y_df
        self.batch_size = batch_size
        self.indexes = np.arange(len(self.x_df))
        self.on_epoch_end()
    
    def __len__(self):
        return len(self.x_df) // self.batch_size
    
    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        
        batch_x = []
        batch_y = []
        for i in indexes:
            filepath = self.x_df.iloc[i]['filepath']
            img = load_img(filepath, target_size=(224, 224))
            img_array = img_to_array(img)
            img_preprocessed = preprocess_input(img_array)
            batch_x.append(img_preprocessed)
            
            if self.y_df is not None:
                label = self.y_df.iloc[i].values
                batch_y.append(label)
        
        batch_x = np.array(batch_x)
        if self.y_df is not None:
            batch_y = np.array(batch_y)
            return batch_x, batch_y
        else:
            return batch_x
    
    def on_epoch_end(self):
        np.random.shuffle(self.indexes)

위의 코드는 Keras의 Sequence를 상속받아 데이터셋을 구현한 ImagesDataset 클래스입니다. 이 클래스는 이미지 데이터를 로드하고 전처리하는 기능을 수행합니다.

ImagesDataset 클래스는 다음과 같은 기능을 제공합니다:

__init__(self, x_df, y_df=None, batch_size=32): 생성자 함수입니다. x_df는 이미지 데이터의 파일 경로를 포함하는 데이터프레임이며, y_df는 해당 이미지의 레이블을 포함하는 데이터프레임입니다. batch_size는 배치 크기를 지정하는 매개변수입니다.

__len__(self): 데이터셋의 배치 개수를 반환합니다.

__getitem__(self, index): 인덱스 index에 해당하는 배치 데이터를 반환합니다. 이미지를 로드하고 전처리하여 배치로 구성된 데이터를 반환합니다. 레이블이 있는 경우에는 이미지와 함께 레이블을 반환합니다.

on_epoch_end(self): 각 에포크가 끝날 때마다 데이터의 인덱스를 섞어줍니다.

이렇게 구현된 ImagesDataset 클래스를 사용하여 데이터셋을 생성한 후, fit 함수를 통해 모델을 훈련시킬 수 있습니다. 위 예시에서는 train_dataset과 eval_dataset을 생성하여 훈련과 평가에 사용하고 있습니다.

train_dataset = ImagesDataset(train_features, train_labels)
eval_dataset = ImagesDataset(eval_features, eval_labels)

model.fit(train_dataset, epochs=10, validation_data=eval_dataset)

위와 같이 데이터셋을 생성하고 모델의 fit 함수를 호출할 때 데이터셋을 전달함으로써 모델이 훈련 데이터셋에서 배치 단위로 데이터를 가져와 훈련을 진행하게 됩니다. 이때 epochs 매개변수는 전체 데이터셋을 몇 번 반복해서 사용하여 훈련할지를 나타내며, validation_data 매개변수에는 평가 데이터셋을 전달하여 훈련 중에 모델의 성능을 평가합니다

위 코드에서는 ImagesDataset 클래스를 사용하여 데이터셋을 구축하였습니다. 이제 해당 데이터셋을 활용하여 모델을 훈련시키는 방법에 대해 설명드리겠습니다.

전체적인 과정은 다음과 같습니다:

데이터셋 구축: ImagesDataset 클래스를 사용하여 훈련 데이터셋과 검증 데이터셋을 생성합니다.

모델 생성: Keras에서 사용할 모델을 생성합니다. 이는 이미지 분류 모델로서, 이미지 데이터를 입력으로 받고 해당 이미지의 클래스를 예측하는 모델입니다.

모델 컴파일: 모델을 컴파일합니다. 이때 손실 함수, 옵티마이저, 평가 지표 등을 설정합니다.

모델 훈련: fit 함수를 사용하여 모델을 훈련시킵니다. 이때 훈련 데이터셋과 검증 데이터셋을 입력으로 전달하며, 훈련 횟수(에포크 수)를 설정합니다.

위의 코드에서 ... 부분에는 실제로 사용할 모델을 구성하는 코드가 들어가야 합니다. 모델 구성에는 여러 가지 방법이 있으며, 예를 들어 이미지 분류를 위한 Convolutional Neural Network (CNN) 모델을 사용할 수 있습니다.

아래는 간단한 예시로서 Sequential 모델을 사용하여 CNN 기반의 이미지 분류 모델을 구성하는 코드입니다.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# 모델 생성
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))  # num_classes는 클래스 개수에 맞게 설정해야 합니다.

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



In [48]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# 1. 데이터셋 구축
train_dataset = ImagesDataset(x_train, y_train)
eval_dataset = ImagesDataset(x_eval, y_eval)

# 2. 모델 생성 CNN 사용
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(9, activation='softmax'))  # num_classes는 클래스 개수에 맞게 설정해야 합니다.

# 3. 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# 4. 모델 훈련
model.fit(train_dataset, epochs=10, validation_data=eval_dataset)

FileNotFoundError: ignored

In [47]:
num_classes = y_train.shape[1]
num_classes

9

모델 컴파일 시 손실 함수로 로그 손실(log loss)을 사용하고자 할 때, loss 매개변수에 'binary_crossentropy' 또는 'categorical_crossentropy'를 지정하면 됩니다. 이 두 가지 손실 함수는 로그 손실을 계산하는 데 사용되며, 다중 클래스 분류와 이진 클래스 분류에 적합합니다.

'binary_crossentropy'는 이진 분류 문제에 사용되는 손실 함수이며, 'categorical_crossentropy'는 다중 클래스 분류 문제에 사용되는 손실 함수입니다. 이 중에 해당하는 손실 함수를 선택하여 모델을 컴파일하면 됩니다.